In [1]:
import wfdb
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import json
import torch
import pandas as pd
import os

In [2]:
from train_ecg_classifier import get_parser, get_df
from rosemary import jpt_parse_args

CUDA_VISIBLE_DEVICES=1 python train_ecg_classifier.py --folder=results/note_ecg_classification/resnet1d  --cuda

cmd = """
--seq_length=4096 \
--folder=results/note_ecg_classification/resnet1d \
--lr=0.001 \
--cuda \
'1' \
'1' \
""".strip()

parser = get_parser()
args = jpt_parse_args(parser, cmd=cmd)
args

/data/vision/polina/shared_software/miniconda3/envs/cxrpe/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Namespace(epochs=70, seed=2, sample_freq=400, seq_length=4096, scale_multiplier=10, batch_size=32, lr=0.001, patience=7, min_lr=1e-07, lr_factor=0.1, net_filter_size=[64, 128, 196, 256, 320], net_seq_lengh=[4096, 1024, 256, 64, 16], dropout_rate=0.8, kernel_size=17, folder='results/note_ecg_classification/resnet1d', traces_dset='tracings', ids_dset='', age_col='age', ids_col=None, cuda=True, n_valid=100, path_to_traces='1', path_to_csv='1')

In [5]:
torch.manual_seed(args.seed)
print(args)
# Set device
device = torch.device('cuda:0' if args.cuda else 'cpu')
folder = args.folder

# Generate output folder if needed
if not os.path.exists(args.folder):
    os.makedirs(args.folder)
# Save config file
with open(os.path.join(args.folder, 'args.json'), 'w') as f:
    json.dump(vars(args), f, indent='\t')


Namespace(epochs=70, seed=2, sample_freq=400, seq_length=4096, scale_multiplier=10, batch_size=32, lr=0.001, patience=7, min_lr=1e-07, lr_factor=0.1, net_filter_size=[64, 128, 196, 256, 320], net_seq_lengh=[4096, 1024, 256, 64, 16], dropout_rate=0.8, kernel_size=17, folder='results/note_ecg_classification/resnet1d', traces_dset='tracings', ids_dset='', age_col='age', ids_col=None, cuda=True, n_valid=100, path_to_traces='1', path_to_csv='1')


In [6]:
from sklearn.model_selection import train_test_split

X = np.stack([x.T for x in ecgs])
y = np.array(df['label'].tolist())
# fill nan with 0
X = np.nan_to_num(X, nan=0.0)
# take the middle portion of the signal
l = (X.shape[2] - args.seq_length)//2
r = l + args.seq_length
X = X[:,:,l:r]
print(X.shape, y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print('after train_test_split: ' X_train.shape, y_train.shape, X_test.shape, y_test.shape)


(3352, 12, 4096) (3352,)
(2681, 12, 4096) (2681,) (671, 12, 4096) (671,)


In [7]:
train_loader = BatchDataloader(X_train, y_train, bs=args.batch_size, mask=np.ones(len(X_train)).astype(bool))
valid_loader = BatchDataloader(X_test, y_test, bs=args.batch_size, mask=np.ones(len(X_test)).astype(bool))
print('train/val loader len: ' len(train_loader), len(valid_loader))

84 21


In [8]:

tqdm.write("Define model...")
N_LEADS = 12  # the 12 leads
N_CLASSES = 1  # just the age
model = ResNet1d(input_dim=(N_LEADS, args.seq_length),
                 blocks_dim=list(zip(args.net_filter_size, args.net_seq_lengh)),
                 n_classes=N_CLASSES,
                 kernel_size=args.kernel_size,
                 dropout_rate=args.dropout_rate)
model.to(device=device)
tqdm.write("Done!")

tqdm.write("Define optimizer...")
optimizer = torch.optim.Adam(model.parameters(), args.lr)
tqdm.write("Done!")

tqdm.write("Define scheduler...")
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=args.patience,
                                                 min_lr=args.lr_factor * args.min_lr,
                                                 factor=args.lr_factor)
tqdm.write("Done!")

Define model...
4096 4096
Done!
Define optimizer...
Done!
Define scheduler...
Done!


In [4]:
from train_ecg_classifier import train, evaluate

ImportError: cannot import name 'train' from 'train_ecg_classifier' (/data/vision/polina/scratch/wpq/github/cxrpe/notebooks/train_ecg_classifier.py)

In [10]:

tqdm.write("Training...")
start_epoch = 0
best_loss = np.Inf
history = pd.DataFrame(columns=['epoch', 'train_loss', 'valid_loss', 'lr',
                                'accuracy', 'auroc', 'precision', 'recall', 
                               ])
for ep in range(start_epoch, args.epochs):
    train_loss = train(ep, train_loader)
    valid_metrics = evaluate(ep, valid_loader)
    valid_loss = valid_metrics['eval_loss']
    # Save best model
    if valid_loss < best_loss:
        # Save model
        torch.save({'epoch': ep,
                    'model': model.state_dict(),
                    'valid_loss': valid_loss,
                    'optimizer': optimizer.state_dict()},
                   os.path.join(folder, 'model.pth'))
        # Update best validation loss
        best_loss = valid_loss
    # Get learning rate
    for param_group in optimizer.param_groups:
        learning_rate = param_group["lr"]
    # Interrupt for minimum learning rate
    if learning_rate < args.min_lr:
        break
    # Save history
    row = pd.DataFrame([{"epoch": ep, "train_loss": train_loss,"valid_loss": valid_loss, "lr": learning_rate,
                     'accuracy': valid_metrics['accuracy'], 'auroc': valid_metrics['auroc'],
                     'precision': valid_metrics['precision'], 'recall': valid_metrics['recall'], }])
    history = pd.concat([history, row], ignore_index=True)
    history.to_csv(os.path.join(folder, 'history.csv'), index=False)
    # Update learning rate
    scheduler.step(valid_loss)
    
    # Print message
    tqdm.write('Epoch {:2d}:  l_train={:.6f}    l_valid={:.6f}    lr={:.7f}\t    acc/auc={:.3f}/{:.3f}'
             .format(ep, train_loss, valid_loss, learning_rate, valid_metrics['accuracy'], valid_metrics['auroc']))
tqdm.write("Done!")

Training...


Epoch  0: valid - Loss: 0.023507: 100%|██████████| 21/21 [00:00<00:00, 44.57it/s]
/tmp/ipykernel_456207/1188268041.py:31: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  history = pd.concat([history, row], ignore_index=True)


Epoch  0:  l_train=0.025641    l_valid=0.023507    lr=0.0010000	    acc/auc=0.496/0.557


Epoch  1: valid - Loss: 0.023695: 100%|██████████| 21/21 [00:00<00:00, 53.86it/s]


Epoch  1:  l_train=0.026264    l_valid=0.023695    lr=0.0010000	    acc/auc=0.474/0.521


...

Epoch 58:  l_train=0.021491    l_valid=0.023920    lr=0.0000001	    acc/auc=0.557/0.605


Epoch 59: valid - Loss: 0.024434: 100%|██████████| 21/21 [00:00<00:00, 49.47it/s]


Epoch 59:  l_train=0.021903    l_valid=0.024434    lr=0.0000001	    acc/auc=0.557/0.605


Epoch 60: valid - Loss: 0.023923: 100%|██████████| 21/21 [00:00<00:00, 51.05it/s]

Done!
